# LangChain Expression Language (LCEL)

# Setup {.smaller}

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
import json

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.schema.runnable import RunnableMap
from langchain.llms import OpenAI

# Simple Chain 


## Create chain{.smaller}

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)

model = ChatOpenAI()

output_parser = StrOutputParser()

In [ ]:
chain = prompt | model | output_parser

## Invoke chain

In [ ]:
chain.invoke({"topic": "a professor at HdM Stuttgar"})

- ''Why did the professor at HdM Stuttgart always carry a ladder?\n\nBecause he wanted to reach new heights in teaching!''


# More complex chain

## Create Vector Store and Retriever


In [ ]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["Yuval Noah Harari is the author of Sapiens", "In A Guide to the Good Life, William Irvine offers a refreshing presentation of Stoicism"],
    embedding=OpenAIEmbeddings()
)

# create a retriever
retriever = vectorstore.as_retriever()

## Retrieve relevant documents {.smaller}

In [ ]:
retriever.get_relevant_documents("who is the author of Sapiens?")

- [Document(page_content='Yuval Noah Harari is the author of Sapiens'),
 Document(page_content='In A Guide to the Good Life, William Irvine offers a refreshing presentation of Stoicism')]



In [ ]:
retriever.get_relevant_documents("Which book did William Irvine write?")

- [Document(page_content='In A Guide to the Good Life, William Irvine offers a refreshing presentation of Stoicism'),
 Document(page_content='Yuval Noah Harari is the author of Sapiens')]


# RAG pipeline


## Create prompt

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

## Runnable Map {.smaller}

- Chain: get user input > fetch relevant context > pass context into prompt > pass into model > pass into output parser to convert into string

- Create dictionary with context and question


In [ ]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

## Invoke chain

In [ ]:
chain.invoke({"question": "who is the author of Sapiens?"})

- 'The author of Sapiens is Yuval Noah Harari.'

## Closer look at the RunnableMap {.smaller}


In [ ]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [ ]:
inputs.invoke({"question": "who is the author of Sapiens?"})

- {'context': [Document(page_content='Yuval Noah Harari is the author of Sapiens'),
  Document(page_content='In A Guide to the Good Life, William Irvine offers a refreshing presentation of Stoicism')],
 'question': 'who is the author of Sapiens?'}


# Bind parameters

## Weather function


In [ ]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

## Bind


In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

## Runnable

In [ ]:
runnable = prompt | model

In [ ]:
runnable.invoke({"input": "what is the weather in sf"})

- AIMessage(content='', additional_kwargs={'function_call': {'name': 'weather_search', 'arguments': '{\n  "airport_code": "SFO"\n}'}})

## Weather and sports search function {.smaller}

In [ ]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

## Bind {.smaller}

In [ ]:
model = model.bind(functions=functions)

In [ ]:
runnable = prompt | model

In [ ]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

- AIMessage(content='', additional_kwargs={'function_call': {'name': 'sports_search', 'arguments': '{\n  "team_name": "patriots"\n}'}})

# Fallbacks

## Use a simple model


In [ ]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="text-davinci-001"
)
simple_chain = simple_model | json.loads

## Input

In [ ]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

## Run simple model

In [ ]:
simple_model.invoke(challenge)

- Note: The next line is expected to fail.

In [ ]:
# simple_chain.invoke(challenge)

- Output is not JSON

## Use different model

In [ ]:
model = ChatOpenAI(temperature=0)

chain = model | StrOutputParser() | json.loads

In [ ]:
chain.invoke(challenge)

- {'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': "Softly it blows, the wind's gentle touch"},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Stone',
  'first_line': 'In moonlit night, a song unheard'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Reed',
  'first_line': 'Shadows sway, a graceful ballet'}}


## New model with fallbacks

In [ ]:
final_chain = simple_chain.with_fallbacks([chain])

In [ ]:
final_chain.invoke(challenge)

- {'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the walls, a secret ballet'}}

# Interface

## Joke example

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

## One input

In [ ]:
chain.invoke({"topic": "professors"})

- 'Why did the professor bring a ladder to the lecture? \n\nBecause they wanted to reach new heights of knowledge!'

## Multiple inputs

In [ ]:
chain.batch([{"topic": "professors"}, {"topic": "students"}])

- ['Why did the professor bring a ladder to class?\n\nBecause they heard the lecture was going to be on high-level concepts!',
 'Why did the student bring a ladder to school?\n\nBecause they wanted to reach for the highest grades!']


## Stream back responses {.smaller}

In [ ]:
for t in chain.stream({"topic": "professors"}):
    print(t)

Why
 did
 the
 professor
 bring
 a
 ladder
 to
 class
?
 


Because
 they
 wanted
 to
 reach
 new
 heights
 in
 education
!

## Async method

In [ ]:
response = await chain.ainvoke({"topic": "professors"})
response